In [ ]:
# Create WebWorker with WebLLM
worker_code = """
import * as webllm from 'https://esm.run/@mlc-ai/web-llm';

let engine = null;

self.onmessage = async function(e) {
    const { type, data, id } = e.data;
    
    if (type === 'init') {
        engine = await webllm.CreateMLCEngine(data.model, {
            initProgressCallback: (progress) => {
                self.postMessage({type: 'progress', data: progress, id});
            }
        });
        self.postMessage({type: 'ready', id});
    } else if (type === 'chat') {
        const response = await engine.chat.completions.create({
            messages: data.messages,
            stream: false
        });
        self.postMessage({
            type: 'response', 
            data: response.choices[0].message.content, 
            id
        });
    }
};
"""

from js import Blob, URL, Worker
blob = Blob.new([worker_code], {"type": "application/javascript"})
worker_url = URL.createObjectURL(blob)
worker = Worker.new(worker_url)

import js
import asyncio

from js import console
import asyncio
from pyodide.ffi import create_proxy

# Dictionary to track pending requests
pending_requests = {}
request_id = 0

def handle_worker_message(event):
    """Handle messages coming back from the worker"""
    data = event.data
    msg_type = data.type
    msg_id = data.id
    
    if msg_id in pending_requests:
        # Resolve the promise for this request
        resolver = pending_requests[msg_id]
        resolver(data)
        del pending_requests[msg_id]

# Create proxy and attach listener
message_handler = create_proxy(handle_worker_message)
worker.addEventListener("message", message_handler)
print("Message listener attached!")

async def send_worker_message(msg_type, data):
    """Send a message to the worker and wait for response"""
    global request_id
    request_id += 1
    msg_id = request_id
    
    # Create a future to wait for the response
    future = asyncio.get_event_loop().create_future()
    pending_requests[msg_id] = lambda result: future.set_result(result)
    
    # Convert Python dict to JavaScript object using to_js()
    from pyodide.ffi import to_js
    
    message = to_js({
        "type": msg_type,
        "data": data,
        "id": msg_id
    }, dict_converter=js.Object.fromEntries)
    
    # Send message to worker
    worker.postMessage(message)
    
    # Wait for response
    response = await future
    return response

# Now initialize the engine
print("Initializing WebLLM engine...")
# model="Llama-3.2-1B-Instruct-q4f32_1-MLC"
model="SmolLM2-360M-Instruct-q4f16_1-MLC"
init_response = await send_worker_message("init", {
    "model": model
})

if init_response.type == "init_complete":
    print(f"✓ {init_response.data}")
elif init_response.type == "error":
    print(f"✗ Error: {init_response.data}")

Message listener attached!
Initializing WebLLM engine...


Initializing WebLLM engine...


<class 'NameError'>: name 'request_id' is not defined